In [1]:
# EX1: Normal Equation by tensorflow.
#-----------------------------------------------#

import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

house = fetch_california_housing()
data_add_1_column = np.c_[np.ones((np.shape(house.data)[0], 1)), house.data]
X = tf.constant(data_add_1_column, dtype=tf.float32, name="X")
y = tf.constant(house.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [20]:
# EX2: Gradient descent by tensflow.
#-----------------------------------------------#

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epoch = 200
learning_rate = 0.01
m, n = house.data.shape

X = tf.constant(data_add_1_column, dtype=tf.float32, name="X")
y = tf.constant(house.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), dtype=tf.float32, name="theta")
y_pre = tf.matmul(X, theta, name="predictions")
erorr = y - y_pre
mse = tf.reduce_mean(tf.square(erorr), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), erorr)
training_op = tf.assign(theta, theta - learning_rate*gradients)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            #print("Epoch:", epoch, "MSE:", mse.eval())
            summary_str = mse_summary.eval()
            file_writer.add_summary(summary_str, epoch)
            sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

file_writer.close()

In [5]:
# EX3: How to caculate gradient in tensorflow.
#-----------------------------------------------#

X_3 = tf.constant([2, 3, 4], dtype=tf.float32, name="X_3")
y_3 = X_3 ** 2 + 2 * X_3
gradients = tf.gradients(y_3, [X_3])[0]
with tf.Session() as sess:
    sess.run(gradients)
    y_3_value = gradients.eval()

print(y_3_value)

[  6.   8.  10.]


In [7]:
# EX4: Placeholder, is a specify node which can feeded anything
#------------------------------------------------#

X_4 = tf.placeholder(dtype=tf.float32)
y_4 = X_4 + 5

with tf.Session() as sess:
    y_4_val = y_4.eval(feed_dict={X_4: [[1, 2, 3]]})

print(y_4_val)

[[ 6.  7.  8.]]


In [26]:
#EX4: modularity
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tensor/chap9/logs/relu1", tf.get_default_graph())

In [25]:
#EX6: Name scope
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    
        b = tf.Variable(0.0, name="bias")                             
        z = tf.add(tf.matmul(X, w), b, name="z")                      
        return tf.maximum(z, 0., name="max")



n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tensor/chap9/logs/relu2", tf.get_default_graph())
file_writer.close()

In [27]:
#EX7: variable scope

reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")
file_writer = tf.summary.FileWriter("tensor/chap9/logs/relu3", tf.get_default_graph())
file_writer.close()

In [28]:
from numpy import *

In [31]:
exp(-9999)

0.0